In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import re
import warnings
warnings.simplefilter('ignore', pd.core.common.SettingWithCopyWarning)

In [ ]:
#各業種銘柄リストへの最初のURLを取得
#get_inds_cwl_urlsで各業種銘柄リストのURLを最後のページまで探索
#get_init_valuesで各業種1ページ分の初期特徴量（code、市場、銘柄名、現在値）を取得する
def get_init_data():
    start_url = 'https://stocks.finance.yahoo.co.jp'
    time.sleep(1)
    response = requests.get(start_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    ind_urls1 = soup.find('ul', attrs={'class': 'blockFloat2Col'})
    ind_urls2 = ind_urls1.find_all('li', attrs={'class': 'ymuiArrow1'})

    #各業種銘柄リストへの最初のURLを取得
    df = pd.DataFrame()
    ind_url_list = []
    for i in ind_urls2:
        ind_url = i.find('a').get('href')
        ind_url_list.append(ind_url)
    for ind_url in ind_url_list:
        get_inds_cwl_urls(ind_url, start_url)
        #if ind_url == 'https://rdsig.yahoo.co.jp/finance/stocks/ids/20d/RV=1/RU=aHR0cHM6Ly9zdG9ja3MuZmluYW5jZS55YWhvby5jby5qcC9zdG9ja3MvcWkvP2lkcz0zMDUw':
            #break


In [ ]:
#各業種銘柄リストのURLを最後のページまで探索する
def get_inds_cwl_urls(url, start_url, count=1):
    time.sleep(1)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('div', attrs={'id': 'listTable'})
    codes = table.find_all('td', attrs={'class': 'center yjM'})
    mkts = table.find_all('td', attrs={'class': 'center yjSt'})
    names = table.find_all('strong', attrs={'class': 'yjMt'})
    prices = table.find_all('div', attrs={'class': 'price yjM'})  
    get_init_values(url)
    
    count += 1
    search_next_url1 = soup.find('div', attrs={'class': 'yjListTab'})
    search_next_url2 = search_next_url1.find_all('a')
    for i in search_next_url2:
        if i.get_text() == str(count):
            next_url = start_url + i.get('href')
            print(next_url)
            get_inds_cwl_urls(next_url, start_url, count)


In [ ]:
#業種1ページ分の初期特徴量（code、市場、銘柄名、現在値）を取得する
def get_init_values(url):
    df_temp = pd.DataFrame()
    time.sleep(1)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    table = soup.find('div', attrs={'id': 'listTable'})
    codes = table.find_all('td', attrs={'class': 'center yjM'})
    mkts = table.find_all('td', attrs={'class': 'center yjSt'})
    names = table.find_all('strong', attrs={'class': 'yjMt'})
    prices = table.find_all('div', attrs={'class': 'price yjM'})
    
    #ズレを防止するため、各特徴量の数が同じか確認する（多分、ズレることはない）
    if len(codes) == len(mkts) == len(names) == len(prices):
        feature_values_dict = {'code':codes, '市場':mkts, '銘柄名':names, '現在値':prices}
        for key, values in feature_values_dict.items():
            codes_list = []
            for value in values:
                codes_list.append(value.get_text())
            df_temp[key] = codes_list
    else:
        print('数が違います')
    global df
    df = pd.concat([df, df_temp], axis=0)


In [ ]:
#初期データの取得
df = pd.DataFrame()
get_init_data()

In [ ]:
#初期データの確認
df.head()

In [ ]:
#初期データの加工→csvに保存
df2 = df

df2 = df2[df2['現在値'] != '---']
df2 = df2.sort_values(by="code")
df2 = df2.reset_index(drop=True)
df2['銘柄名'] = df2['銘柄名'].str.replace('\(株\)', '')
df2['現在値'] = df2['現在値'].str.replace(',', '')

df2.to_csv('全銘柄データ（初期）.csv', mode='w', index=False, encoding="shift-jis")
df2.dtypes

In [3]:
#初期データの読み込み、特徴量の作成
df3 = pd.read_csv('全銘柄データ（初期）.csv', encoding=("shift-jis"), header=0)
df3.dtypes
feature2_list = ['業種', '出来高', '配当利回り', '予PER', '実PBR',
                '単元株数', '売上高', '経常利益', '自己資本比率', 
                '実ROE', '優待月(上)', '優待月(下)', '単元倍率', 
                '特色', '連結事業']
for feature2 in feature2_list:
    df3[feature2] = ''
df3

,code,市場,銘柄名,現在値,業種,出来高,配当利回り,予PER,実PBR,単元株数,売上高,経常利益,自己資本比率,実ROE,優待月(上),優待月(下),単元倍率,特色,連結事業
0,1301,東証1部,極洋,3015.0,,,,,,,,,,,,,,,
1,1332,東証1部,日本水産,533.0,,,,,,,,,,,,,,,
2,1333,東証1部,マルハニチロ,2572.0,,,,,,,,,,,,,,,
3,1352,東証1部,ホウスイ,967.0,,,,,,,,,,,,,,,
4,1375,東証1部,雪国まいたけ,1823.0,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3826,9993,東証1部,ヤマザワ,1704.0,,,,,,,,,,,,,,,
3827,9994,東証1部,やまや,2278.0,,,,,,,,,,,,,,,
3828,9995,東証1部,グローセル,416.0,,,,,,,,,,,,,,,
3829,9996,東証JQS,サトー商会,1514.0,,,,,,,,,,,,,,,


In [52]:
#銘柄のページごとに残りの特徴量を取得する
def get_second_values(start_url, code):
    #time.sleep(1)
    response = requests.get(start_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    
    ###銘柄トップページの値を抽出
    #トップの「業種」を抽出
    industry_pre = soup.find('div', attrs={'class': '_23Jev3qx'})
    industry = industry_pre.find('a').get_text()

    #上の表に入っている「出来高」を抽出
    volume_pre1 = soup.find_all('dl', attrs={'class': '_38iJU1zx _3Quzw23Q'})
    volume_pre2 = [s for s in volume_pre1 if '出来高' in s.text]
    volume = volume_pre2[0].find('span', attrs={'class': '_3rXWJKZF _11kV6f2G'}).get_text()

    #下の表に入っている「配当利回り」「PER」「PBR」「単元株数」を抽出
    elements = soup.find_all('dl', attrs={'class': '_38iJU1zx _2pSv51JU'})

    div_eld_pre = [s for s in elements if '配当利回り' in s.text]
    div_eld = div_eld_pre[0].find('span', attrs={'class': '_3rXWJKZF _11kV6f2G'}).get_text()

    per_pre = [s for s in elements if 'PER' in s.text]
    per = per_pre[0].find('span', attrs={'class': '_3rXWJKZF _11kV6f2G'}).get_text()

    pbr_pre = [s for s in elements if 'PBR' in s.text]
    pbr = pbr_pre[0].find('span', attrs={'class': '_3rXWJKZF _11kV6f2G'}).get_text()

    unit_pre = [s for s in elements if '単元株数' in s.text]
    unit = unit_pre[0].find('span', attrs={'class': '_3rXWJKZF _11kV6f2G'}).get_text()

    
    ###企業情報ページの値を抽出（ページを二回移動して、連結決算推移のページに行く）
    cpinfo_pre_url = soup.find_all('a', text='企業情報')[1].get('href')
    #time.sleep(1)
    response2 = requests.get(cpinfo_pre_url)
    soup2 = BeautifulSoup(response2.content, 'html.parser')
    connect_url = 'https://profile.yahoo.co.jp'
    cpinfo_url = connect_url + soup2.find('a', text='連結決算推移').get('href')
    #time.sleep(1)
    response3 = requests.get(cpinfo_url)
    soup3 = BeautifulSoup(response3.content, 'html.parser')
    
    #連結決算が無い企業は、単独決算を抽出する
    if '月期' not in soup3.text:
        cpinfo_url = connect_url + soup2.find('a', text='単独決算推移').get('href')
        #time.sleep(1)
        response3 = requests.get(cpinfo_url)
        soup3 = BeautifulSoup(response3.content, 'html.parser')
        
    #「売上高」「経常利益」「自己資本比率」「ROE」を抽出
    elements2 = soup3.find_all('tr', attrs={'bgcolor': '#ffffff'})
    sales_pre = [s for s in elements2 if '売上高' in s.text]
    sales = sales_pre[0].find('td', attrs={'align': 'right'}).get_text().replace('百万円', '')

    income_pre = [s for s in elements2 if '経常利益' in s.text]
    income = income_pre[0].find('td', attrs={'align': 'right'}).get_text().replace('百万円', '')

    cpratio_pre = [s for s in elements2 if '自己資本比率' in s.text]
    cpratio = cpratio_pre[0].find('td', attrs={'align': 'right'}).get_text().replace('%', '')

    roe_pre = [s for s in elements2 if 'ROE（自己資本利益率）' in s.text]
    roe = roe_pre[0].find('td', attrs={'align': 'right'}).get_text().replace('%', '')


    ###企業情報ページの値を抽出
    stockholder_url = soup.find_all('a', text='株主優待')[1].get('href')
    #time.sleep(1)
    response4 = requests.get(stockholder_url)
    soup4 = BeautifulSoup(response4.content, 'html.parser')
    
    #「優待月㊤」「優待月㊦」を抽出
    if '株主優待情報はありません' in soup4.text:
        right_m_1st = ''
        right_m_2nd = ''
        unit_num = ''
    else:
        right_m = soup4.find('td', attrs={'class': '_2YqTjDvt'}).get_text()
        right_m = re.sub('月.*?日', '', right_m)
        right_m_list = sorted(list(map(int, right_m.split('・'))))
        #優待が年4回ある場合（3，6，9，12のみ）、上期を3に下期を9にする
        if len(right_m_list) == 4:
            right_m_1st = right_m_list[0]
            right_m_2nd = right_m_list[2]
        elif len(right_m_list) == 2:
            right_m_1st = right_m_list[0]
            right_m_2nd = right_m_list[1] 
        elif len(right_m_list) == 1:
            if right_m_list[0] <= 6:
                right_m_1st = right_m_list[0]
                right_m_2nd = ''
            else:
                right_m_1st = ''
                right_m_2nd = right_m_list[0]
        else:
            print('特殊な形態　要確認！')
        
        #「単元倍率」を抽出
        unit_num_pre1 = soup4.find('table', attrs={'class': 'zoW_RJcx'})
        unit_num_pre2 = unit_num_pre1.find_all('td')[1].get_text().replace('株', '')
        unit_num = int(unit_num_pre2) / int(unit)

    
    ###企業情報ページの値を抽出
    #「特色」「連結事業」を抽出
    elements3 = soup2.find_all('tr', attrs={'bgcolor': '#ffffff'})
    char_pre = [s for s in elements3 if '特色' in s.text]
    char = char_pre[0].find('td', attrs={'colspan': '3'}).get_text()

    cons_viz_pre = char_pre = [s for s in elements3 if '連結事業' in s.text]
    cons_viz = cons_viz_pre[0].find('td', attrs={'colspan': '3'}).get_text().replace('【連結事業】', '')
    
    
    ###データフレームに抽出した値を記入
    feature_values2_dict = {'業種':industry, '出来高':volume, '配当利回り':div_eld, '予PER':per, '実PBR':pbr,
                           '単元株数':unit, '売上高':sales, '経常利益':income, '自己資本比率':cpratio, 
                           '実ROE':roe, '優待月(上)':right_m_1st, '優待月(下)':right_m_2nd, '単元倍率':unit_num,
                           '特色':char, '連結事業':cons_viz,}

    for key, value in feature_values2_dict.items():
        df3[key][df3['code']==code] = value

In [7]:
#for i in range(len(df3)):
for i in range(30):
    code = df3.iloc[i, 0]
    if df3.iloc[i, 1] == '名古屋セ' or df3.iloc[i, 1] == '名証1部' or df3.iloc[i, 1] == '名証2部' :
        section = 'N'
    elif df3.iloc[i, 1] == '札証' or df3.iloc[i, 1] == '札幌ア':
        section = 'S'
    elif df3.iloc[i, 1] == '福証' or df3.iloc[i, 1] == '福Q':
        section = 'F' 
    else:
        section = 'T'
    start_url = 'https://finance.yahoo.co.jp/quote/{}.{}'.format(code, section)
    print(start_url)
    get_second_values(start_url, code)
    #time.sleep(1)


https://finance.yahoo.co.jp/quote/1301.T
https://finance.yahoo.co.jp/quote/1332.T
https://finance.yahoo.co.jp/quote/1333.T
https://finance.yahoo.co.jp/quote/1352.T
https://finance.yahoo.co.jp/quote/1375.T
https://finance.yahoo.co.jp/quote/1376.T
https://finance.yahoo.co.jp/quote/1377.T
https://finance.yahoo.co.jp/quote/1379.T
https://finance.yahoo.co.jp/quote/1380.T
https://finance.yahoo.co.jp/quote/1381.T
https://finance.yahoo.co.jp/quote/1382.T
https://finance.yahoo.co.jp/quote/1383.T
https://finance.yahoo.co.jp/quote/1384.T
https://finance.yahoo.co.jp/quote/1400.T
https://finance.yahoo.co.jp/quote/1401.T
https://finance.yahoo.co.jp/quote/1407.T
https://finance.yahoo.co.jp/quote/1413.T
https://finance.yahoo.co.jp/quote/1414.T
https://finance.yahoo.co.jp/quote/1417.T
https://finance.yahoo.co.jp/quote/1418.T
https://finance.yahoo.co.jp/quote/1419.T
https://finance.yahoo.co.jp/quote/1420.T
https://finance.yahoo.co.jp/quote/1429.T
https://finance.yahoo.co.jp/quote/1430.T
https://finance.

In [62]:
#csvに保存
df3.to_csv('全銘柄データ（加工前）.csv', mode='w', index=False, encoding="shift-jis")
df3

,code,市場,銘柄名,現在値,業種,出来高,配当利回り,予PER,実PBR,単元株数,売上高,経常利益,自己資本比率,実ROE,優待月(上),優待月(下),単元倍率,特色,連結事業
0,1301,東証1部,極洋,3015.0,水産・農林業,"25,000",2.65,9.27,0.80,100,249197,4879,34.7,10.50,3,,1,水産品の貿易、加工、買い付け主力。すしネタに強み。加工食品は業務用が軸。海外加工比率高い,水産商事48(1)、冷凍食品32(1)、常温食品7(4)、物流サービス0(11)、鰹・鮪11...
1,1332,東証1部,日本水産,533.0,水産・農林業,"718,300",1.79,11.04,0.98,100,656491,22750,35.7,8.96,3,,5,水産大手。冷食など食品比重大。国内・南米で養殖。ＥＰＡなど化成品でも実力。傘下に日水製薬,水産42(4)、食品49(4)、ファイン4(10)、物流2(8)、他3(2)【海外】31(2...
2,1333,東証1部,マルハニチロ,2572.0,水産・農林業,"82,800",1.55,9.67,0.95,100,862585,18130,26.8,4.20,3,,1,水産最大手。国内外に強固な流通網持つ。冷食・缶詰など加工食品も大手。畜産商事や化成品も,漁業・養殖4(-1)、商事48(1)、海外18(2)、加工26(3)、物流2(9)、他2(7...
3,1352,東証1部,ホウスイ,967.0,卸売業,100,2.10,11.41,1.14,100,83265,1163,16.9,11.95,,,,中央魚類の子会社。鮮魚、冷凍魚などの卸売り、冷蔵倉庫を主体とする水産物の総合流通企業,冷蔵倉庫9(6)、水産食品91(1)(2020.3)
4,1375,東証1部,雪国まいたけ,1823.0,水産・農林業,"78,300",2.33,15.68,7.78,100,51380,7125,25.9,67.14,3,,1,主力のマイタケ軸にキノコ製品を量産。小売業者への直売中心。親会社の神明ＨＤと西日本開拓,まいたけ57、エリンギ10、ぶなしめじ19、茸他8、他5(2020.3)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3826,9993,東証1部,ヤマザワ,1704.0,,,,,,,,,,,,,,,
3827,9994,東証1部,やまや,2278.0,,,,,,,,,,,,,,,
3828,9995,東証1部,グローセル,416.0,,,,,,,,,,,,,,,
3829,9996,東証JQS,サトー商会,1514.0,,,,,,,,,,,,,,,


In [63]:
#csvにから読出し→データの加工・順番の並び替え→csvに保存
df4 = pd.read_csv('全銘柄データ（加工前）.csv', encoding=("shift-jis"), header=0)
df4['利益/売上'] = ''
df4['利益/売上'][df4['経常利益'] != '' ] = df4['経常利益']  / df4['売上高']
df4 = df4.reindex(columns=['code', '銘柄名', '市場', '業種', '単元株数', '現在値',  '出来高', '配当利回り', 
                     '予PER', '実PBR', '実ROE', '売上高', '経常利益', '利益/売上', '自己資本比率',
                     '優待月(上)', '優待月(下)', '単元倍率', '特色', '連結事業'])
df4

,code,銘柄名,市場,業種,単元株数,現在値,出来高,配当利回り,予PER,実PBR,実ROE,売上高,経常利益,利益/売上,自己資本比率,優待月(上),優待月(下),単元倍率,特色,連結事業
0,1301,極洋,東証1部,水産・農林業,100.0,3015.0,"25,000",2.65,9.27,0.80,10.50,249197.0,4879.0,0.0195789,34.7,3.0,NaN,1.0,水産品の貿易、加工、買い付け主力。すしネタに強み。加工食品は業務用が軸。海外加工比率高い,水産商事48(1)、冷凍食品32(1)、常温食品7(4)、物流サービス0(11)、鰹・鮪11...
1,1332,日本水産,東証1部,水産・農林業,100.0,533.0,"718,300",1.79,11.04,0.98,8.96,656491.0,22750.0,0.0346539,35.7,3.0,NaN,5.0,水産大手。冷食など食品比重大。国内・南米で養殖。ＥＰＡなど化成品でも実力。傘下に日水製薬,水産42(4)、食品49(4)、ファイン4(10)、物流2(8)、他3(2)【海外】31(2...
2,1333,マルハニチロ,東証1部,水産・農林業,100.0,2572.0,"82,800",1.55,9.67,0.95,4.20,862585.0,18130.0,0.0210182,26.8,3.0,NaN,1.0,水産最大手。国内外に強固な流通網持つ。冷食・缶詰など加工食品も大手。畜産商事や化成品も,漁業・養殖4(-1)、商事48(1)、海外18(2)、加工26(3)、物流2(9)、他2(7...
3,1352,ホウスイ,東証1部,卸売業,100.0,967.0,100,2.10,11.41,1.14,11.95,83265.0,1163.0,0.0139675,16.9,NaN,NaN,NaN,中央魚類の子会社。鮮魚、冷凍魚などの卸売り、冷蔵倉庫を主体とする水産物の総合流通企業,冷蔵倉庫9(6)、水産食品91(1)(2020.3)
4,1375,雪国まいたけ,東証1部,水産・農林業,100.0,1823.0,"78,300",2.33,15.68,7.78,67.14,51380.0,7125.0,0.138673,25.9,3.0,NaN,1.0,主力のマイタケ軸にキノコ製品を量産。小売業者への直売中心。親会社の神明ＨＤと西日本開拓,まいたけ57、エリンギ10、ぶなしめじ19、茸他8、他5(2020.3)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3826,9993,ヤマザワ,東証1部,NaN,NaN,1704.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3827,9994,やまや,東証1部,NaN,NaN,2278.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3828,9995,グローセル,東証1部,NaN,NaN,416.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3829,9996,サトー商会,東証JQS,NaN,NaN,1514.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
df4.to_csv('全銘柄データ（完成）.csv', mode='w', index=False, encoding="shift-jis")

In [ ]:
start_url = 'https://finance.yahoo.co.jp/quote/1401.T'
response = requests.get(start_url)
soup = BeautifulSoup(response.content, 'html.parser')

###銘柄トップページの値を抽出
#トップの「業種」を抽出
industry_pre = soup.find('div', attrs={'class': '_23Jev3qx'})
industry = industry_pre.find('a').get_text()

#上の表に入っている「出来高」を抽出
volume_pre1 = soup.find_all('dl', attrs={'class': '_38iJU1zx _3Quzw23Q'})
volume_pre2 = [s for s in volume_pre1 if '出来高' in s.text]
volume = volume_pre2[0].find('span', attrs={'class': '_3rXWJKZF _11kV6f2G'}).get_text()

#下の表に入っている「配当利回り」「PER」「PBR」「単元株数」を抽出
elements = soup.find_all('dl', attrs={'class': '_38iJU1zx _2pSv51JU'})

div_eld_pre = [s for s in elements if '配当利回り' in s.text]
div_eld = div_eld_pre[0].find('span', attrs={'class': '_3rXWJKZF _11kV6f2G'}).get_text()

per_pre = [s for s in elements if 'PER' in s.text]
per = per_pre[0].find('span', attrs={'class': '_3rXWJKZF _11kV6f2G'}).get_text()

pbr_pre = [s for s in elements if 'PBR' in s.text]
pbr = pbr_pre[0].find('span', attrs={'class': '_3rXWJKZF _11kV6f2G'}).get_text()

unit_pre = [s for s in elements if '単元株数' in s.text]
unit = unit_pre[0].find('span', attrs={'class': '_3rXWJKZF _11kV6f2G'}).get_text()


###企業情報ページの値を抽出（ページを二回移動して、連結決算推移のページに行く）
cpinfo_pre_url = soup.find_all('a', text='企業情報')[1].get('href')
response2 = requests.get(cpinfo_pre_url)
soup2 = BeautifulSoup(response2.content, 'html.parser')
connect_url = 'https://profile.yahoo.co.jp'
cpinfo_url = connect_url + soup2.find('a', text='連結決算推移').get('href')
response3 = requests.get(cpinfo_url)
soup3 = BeautifulSoup(response3.content, 'html.parser')

#連結決算が無い企業は、単独決算を抽出する
if '月期' not in soup3.text:
    cpinfo_url = connect_url + soup2.find('a', text='単独決算推移').get('href')
    response3 = requests.get(cpinfo_url)
    soup3 = BeautifulSoup(response3.content, 'html.parser')
    
#「売上高」「経常利益」「自己資本比率」「ROE」を抽出
elements2 = soup3.find_all('tr', attrs={'bgcolor': '#ffffff'})
sales_pre = [s for s in elements2 if '売上高' in s.text]
sales = sales_pre[0].find('td', attrs={'align': 'right'}).get_text().replace('百万円', '')

income_pre = [s for s in elements2 if '経常利益' in s.text]
income = income_pre[0].find('td', attrs={'align': 'right'}).get_text().replace('百万円', '')

cpratio_pre = [s for s in elements2 if '自己資本比率' in s.text]
cpratio = cpratio_pre[0].find('td', attrs={'align': 'right'}).get_text().replace('%', '')

roe_pre = [s for s in elements2 if 'ROE（自己資本利益率）' in s.text]
roe = roe_pre[0].find('td', attrs={'align': 'right'}).get_text().replace('%', '')


###企業情報ページの値を抽出
stockholder_url = soup.find_all('a', text='株主優待')[1].get('href')
response4 = requests.get(stockholder_url)
soup4 = BeautifulSoup(response4.content, 'html.parser')

#「優待月㊤」「優待月㊦」を抽出
if '株主優待情報はありません' in soup4.text:
    right_m_1st = ''
    right_m_2nd = ''
    unit_num = ''
    print('sssssssssssssssss')
else:
    right_m = soup4.find('td', attrs={'class': '_2YqTjDvt'}).get_text()
    right_m = re.sub('月.*?日', '', right_m)
    right_m_list = sorted(list(map(int, right_m.split('・'))))
    #優待が年4回ある場合（3，6，9，12のみ）、上期を3に下期を9にする
    if len(right_m_list) == 4:
        right_m_1st = right_m_list[0]
        right_m_2nd = right_m_list[2]
    elif len(right_m_list) == 2:
        right_m_1st = right_m_list[0]
        right_m_2nd = right_m_list[1] 
    elif len(right_m_list) == 1:
        if right_m_list[0] <= 6:
            right_m_1st = right_m_list[0]
            right_m_2nd = ''
        else:
            right_m_1st = ''
            right_m_2nd = right_m_list[0]
    else:
        print('特殊な形態　要確認！')

    #「単元倍率」を抽出
    unit_num_pre1 = soup4.find('table', attrs={'class': 'zoW_RJcx'})
    unit_num_pre2 = unit_num_pre1.find_all('td')[1].get_text().replace('株', '')
    unit_num = int(unit_num_pre2) / int(unit)


###企業情報ページの値を抽出
#「特色」「連結事業」を抽出
elements3 = soup2.find_all('tr', attrs={'bgcolor': '#ffffff'})
char_pre = [s for s in elements3 if '特色' in s.text]
char = char_pre[0].find('td', attrs={'colspan': '3'}).get_text()

cons_viz_pre = char_pre = [s for s in elements3 if '連結事業' in s.text]
cons_viz = cons_viz_pre[0].find('td', attrs={'colspan': '3'}).get_text().replace('【連結事業】', '')

In [ ]:
#「出来高」「配当利回り」「PER」「PBR」「単元株数」「売上高」「経常利益」「自己資本比率」「ROE」
#「優待月㊤」「優待月㊦」「単元倍率」
warnings.simplefilter('ignore', pd.core.common.SettingWithCopyWarning)
#pd.set_option('display.max_rows', 50)

feature_values2_dict = {'業種':industry, '出来高':volume, '配当利回り':div_eld, '予PER':per, '実PBR':pbr,
                       '単元株数':unit, '売上高':sales, '経常利益':income, '自己資本比率':cpratio, 
                       '実ROE':roe, '優待月(上)':right_m_1st, '優待月(下)':right_m_2nd, '単元倍率':unit_num,
                       '特色':char, '連結事業':cons_viz,}


for key, value in feature_values2_dict.items():
    df3[key][df3['code']==1401] = value
feature_values2_dict.keys()
df3.head(50)

In [ ]:
import re

s = '6月末日・3月末日・9月末日・12月末日'
s = re.sub('月.*?日', '', s)
num_list = sorted(list(map(int, s.split('・'))))
len(num_list)

#print(re.sub('月.*?日', '', s))

In [ ]:
import pandas as pd
columns = ['code', '銘柄名', '市場', '業種', '現在値', '出来高', '配当利回り',
           '予PER', '実PBR', '実POE', '売上（実）', '利益（実）', '売上/利益', '自己資本比率']
df = pd.DataFrame(columns=columns)
df

In [41]:
#データの加工→csvに保存
df4 = df3.copy()

#df4['経常利益'] = df4['経常利益'].str.replace(',', '')
#df4['売上高'] = df4['売上高'].str.replace(',', '')
df4['経常利益'][df4['経常利益'] == '' ] = ''
#df4['経常利益'] = df4['経常利益'].astype(np.int32)
#df4['経常利益'][df4['経常利益']==''] = 333

df4['利益/売上'] = ''
df4['利益/売上'][df4['経常利益'] != '' ] = df4['経常利益']  / df4['売上高']

df4.to_csv('全銘柄データ（加工前）.csv', mode='w', index=False, encoding="shift-jis")
df4.dtypes
df4

,code,市場,銘柄名,現在値,業種,出来高,配当利回り,予PER,実PBR,単元株数,売上高,経常利益,自己資本比率,実ROE,優待月(上),優待月(下),単元倍率,特色,連結事業,利益/売上
0,1301,東証1部,極洋,3015.0,水産・農林業,"25,000",2.65,9.27,0.80,100,249197,4879,34.7,10.50,3,,1,水産品の貿易、加工、買い付け主力。すしネタに強み。加工食品は業務用が軸。海外加工比率高い,水産商事48(1)、冷凍食品32(1)、常温食品7(4)、物流サービス0(11)、鰹・鮪11...,
1,1332,東証1部,日本水産,533.0,水産・農林業,"718,300",1.79,11.04,0.98,100,656491,22750,35.7,8.96,3,,5,水産大手。冷食など食品比重大。国内・南米で養殖。ＥＰＡなど化成品でも実力。傘下に日水製薬,水産42(4)、食品49(4)、ファイン4(10)、物流2(8)、他3(2)【海外】31(2...,
2,1333,東証1部,マルハニチロ,2572.0,水産・農林業,"82,800",1.55,9.67,0.95,100,862585,18130,26.8,4.20,3,,1,水産最大手。国内外に強固な流通網持つ。冷食・缶詰など加工食品も大手。畜産商事や化成品も,漁業・養殖4(-1)、商事48(1)、海外18(2)、加工26(3)、物流2(9)、他2(7...,
3,1352,東証1部,ホウスイ,967.0,卸売業,100,2.10,11.41,1.14,100,83265,1163,16.9,11.95,,,,中央魚類の子会社。鮮魚、冷凍魚などの卸売り、冷蔵倉庫を主体とする水産物の総合流通企業,冷蔵倉庫9(6)、水産食品91(1)(2020.3),
4,1375,東証1部,雪国まいたけ,1823.0,水産・農林業,"78,300",2.33,15.68,7.78,100,51380,7125,25.9,67.14,3,,1,主力のマイタケ軸にキノコ製品を量産。小売業者への直売中心。親会社の神明ＨＤと西日本開拓,まいたけ57、エリンギ10、ぶなしめじ19、茸他8、他5(2020.3),
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3826,9993,東証1部,ヤマザワ,1704.0,,,,,,,,,,,,,,,,
3827,9994,東証1部,やまや,2278.0,,,,,,,,,,,,,,,,
3828,9995,東証1部,グローセル,416.0,,,,,,,,,,,,,,,,
3829,9996,東証JQS,サトー商会,1514.0,,,,,,,,,,,,,,,,
